# Prepare Evaluation Data

This notebook will help you set up an input file for running an evaluation using the [ScoreAnswers Notebook](ScoreAnswers.ipynb) (or in Azure).

It will take a list of questions and get responses through DC API's chat endpoint

- Input: A CSV file with a `question` and `ground_truth` column 
  - optionally a `context` column (for Azure evaluations)
  - Note that for help in generating initial ground truths you can run a csv with just questions and then rename the answer column
- Output: A `CSV` and a `JSONL` file which additionally has an `answer` field


## Prerequisite: Setup Environment Variables

- Save these two files to the current directory:  [.env.staging](https://github.com/nulib/miscellany/blob/main/chat-eval/.env.staging) and [.env.production](https://github.com/nulib/miscellany/blob/main/chat-eval/.env.production)
- [Login to DCAPI](https://dcapi.rdc-staging.library.northwestern.edu/api/v2/auth/login?goto=https://dcapi.rdc-staging.library.northwestern.edu/api/v2/auth/token?ttl=604800) and copy the token value
- replace `DC_API_TOKEN` value in both files 
- Token expires in 1 week. After updating your `.env` file you will need to __**restart the kernel**__


## Setup the Environment

First start by importing and setting up the libraries we need:

In [1]:
#install required packages
%pip install pandas
%pip install requests
%pip install python-dotenv
%pip install ipywidgets
%pip install ipython

# import required packages
import pandas as pd
import os
import random
import json, requests
from datetime import datetime
from dotenv import load_dotenv
import ipywidgets as widgets
import io
from IPython.display import display

  Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 4.2 MB/s eta 0:00:004.2 MB/s eta 0:00:01
Using cached numpy-2.1.1-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py

## Select your environment

Use the dropdown to select DCAPI's staging or production chat endpoint. (use staging unless you have a reason to use prod)

In [2]:
# run the cell, then use the dropdown to select your environment)
env_to_use = widgets.Dropdown(
    options=[('staging', '.env.staging'), ('production', '.env.production')],
    description='Environment:',
)

display(env_to_use)

Dropdown(description='Environment:', options=(('staging', '.env.staging'), ('production', '.env.production')),…

## Make sure you're logged in

In [3]:
# load environment variables
load_dotenv(env_to_use.value)

DC_CHAT_URL = os.getenv('DC_CHAT_URL')
DC_API_TOKEN = os.getenv('DC_API_TOKEN')
DC_API_WHOAMI = os.getenv('DC_API_WHOAMI')

whoami_reponse = requests.get(DC_API_WHOAMI, params=None, headers={'Authorization': 'Bearer ' + DC_API_TOKEN})

# confirm environment variables loaded correctly
if (whoami_reponse.json().get('isLoggedIn') == True):
    print("You are logged in as " + whoami_reponse.json().get('sub') + " using " + env_to_use.label + " environment.")
else:
    print("ERROR: Please check your API token.")

You are logged in as ksd927 using staging environment.


## Configure your input file and load data

Setup the input file name and make sure it is readable

In [4]:
# Run cell, then use the file upload widget to your input file
uploader = widgets.FileUpload(
    accept='csv', 
    multiple=False  
)

display(uploader)

FileUpload(value=(), accept='csv', description='Upload')

In [8]:
# read the input file
uploaded_file = uploader.value[0]
questions = pd.read_csv(io.BytesIO(uploaded_file.content))

# store the input filename in a variable
input_filename = uploaded_file.name

# preview the input file
questions.head()

,question,ground_truth
0,How did World War II propaganda posters influe...,World War II propaganda posters played a cruci...
1,What contributions did Achille Paganini make t...,The provided documents do not contain informat...
2,What role did Larry Hanks play in the folk mus...,Larry Hanks was an active participant in the f...
3,What were the key policies of Murtala Muhammed...,Murtala Muhammed's government in Nigeria focus...
4,How has the political map of Africa changed si...,The political map of Africa has undergone sign...


## Load helper functions

Run the cell so helper functions are loaded

In [6]:
# Setup: Functions to get an answer to the question

def format_answer(response, with_context=False):
    if with_context:
        return pd.Series([response['answer'], response['context']])
    else:
        return response['answer']
    
def format_error(with_context=False):
    if with_context:
        return pd.Series(["--ERROR--", "--ERROR--"])
    else:
        return "--ERROR--"

def get_answer(question, with_context=False):
    url = DC_CHAT_URL
    header = {'Content-Type': 'application/json'}
    
    body = {
        'message': 'chat',
        'auth': DC_API_TOKEN,
        'ref': 'DEV-TEAM-TEST-' + str(random.random()),
        "question": question
    }
    print("Asking question: " + question)
    
    
    try:
        response = requests.post(url, json.dumps(body), headers=header)
        response.raise_for_status()
        print(f"Response: {response.status_code}")
        if response.status_code != 200:
            print('Status:', response.status_code, response.reason)
            return format_error(with_context)
        response_json = response.json()
        return format_answer(response_json, with_context)
    except Exception as err:
        print(f"Other error occurred: {err}")
        return format_error(with_context)
    
def get_answers(questions, with_context=False):
    if with_context:
       questions[['answer', 'context']] = questions['question'].apply(lambda x:get_answer(x, with_context))
    else:
        questions['answer'] = questions['question'].apply(lambda x:get_answer(x, with_context))
        
    print("Done")
    return questions

## Generate the answers from DCAPI

Configure `with_context` to whether you want to fetch context along with the answers

Run the below to fetch answers (and optionally context)

In [9]:
# Run to getenerate answers (will take some time)

# Set with_context to True if you want to get the context column along with the answer
# (Needed for some of the Azure evaluations)
with_context = False

# get answers
get_answers(questions, with_context)

# preview answers
questions.head()


Asking question: How did World War II propaganda posters influence public opinion and morale during the war?
Response: 200
Asking question: What contributions did Achille Paganini make to music, and how did his work influence later composers?
Response: 200
Asking question: What role did Larry Hanks play in the folk music revival, and what are some of his most influential performances?
Response: 200
Asking question: What were the key policies of Murtala Muhammed's government in Nigeria, and how did his leadership impact the country?
Response: 200
Asking question: How has the political map of Africa changed since the 19th century, and what historical events have driven these changes?
Response: 200
Done


,question,ground_truth,answer
0,How did World War II propaganda posters influe...,World War II propaganda posters played a cruci...,World War II propaganda posters played a cruci...
1,What contributions did Achille Paganini make t...,The provided documents do not contain informat...,The documents provided do not explicitly menti...
2,What role did Larry Hanks play in the folk mus...,Larry Hanks was an active participant in the f...,Larry Hanks was a significant figure in the fo...
3,What were the key policies of Murtala Muhammed...,Murtala Muhammed's government in Nigeria focus...,Murtala Muhammed's government in Nigeria focus...
4,How has the political map of Africa changed si...,The political map of Africa has undergone sign...,The political map of Africa has undergone sign...


## Write the results to file

It will write both `CSV` and `JSONL` files. (`JSONL` seems to be a little less buggy in Azure but YMMV)

In [10]:
# write the output files
timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
os.makedirs(os.path.join('output_files', timestamp), exist_ok=True)
output_base_path = f"output_files/{timestamp}"
jsonl_filename = os.path.join(output_base_path, f"{os.path.splitext(os.path.basename(input_filename))[0]}.jsonl")

outJson = questions.to_json(orient="records", lines=True) 
with open(jsonl_filename, 'w') as outfile:
    outfile.write(outJson)

csv_filename = os.path.join(output_base_path, f"{os.path.splitext(os.path.basename(input_filename))[0]}.csv")
questions.to_csv(csv_filename, index=False)

print(f"Output files saved to: {jsonl_filename} and {csv_filename}")

Output files saved to: output_files/20240920081520/5_realistic_with_ground_truth.jsonl and output_files/20240920081520/5_realistic_with_ground_truth.csv
